In [0]:
dbutils.fs.rm("/FileStore/tables/scd2demo", recurse=True)

Out[2]: True

In [0]:
%sql
DROP TABLE IF EXISTS scd2demo;
CREATE OR REPLACE TABLE scd2demo (
  pk1 INT,
  pk2 STRING,
  dim1 INT,
  dim2 INT,
  dim3 INT,
  dim4 INT,
  active_status STRING,
  start_date TIMESTAMP,
  end_date TIMESTAMP
)
USING DELTA
LOCATION '/FileStore/tables/scd2demo'

In [0]:
%sql
INSERT INTO scd2demo VALUES
  (111, 'Unit1', 200, 500, 800, 400, 'Y', current_timestamp(), '9999-12-31'),
  (222, 'Unit2', 900, NULL, 700, 100, 'Y', current_timestamp(), '9999-12-31'),
  (333, 'Unit3', 300, 900, 250, 650, 'Y', current_timestamp(), '9999-12-31')

num_affected_rows,num_inserted_rows
3,3


In [0]:
from delta import *
targetTable = DeltaTable.forPath(spark, '/FileStore/tables/scd2demo')
targetDF = targetTable.toDF()
targetDF.display()

pk1,pk2,dim1,dim2,dim3,dim4,active_status,start_date,end_date
111,Unit1,200,500,800,400,Y,2025-11-29T15:48:33.339+0000,9999-12-31T00:00:00.000+0000
222,Unit2,900,null,700,100,Y,2025-11-29T15:48:33.339+0000,9999-12-31T00:00:00.000+0000
333,Unit3,300,900,250,650,Y,2025-11-29T15:48:33.339+0000,9999-12-31T00:00:00.000+0000


### Create an Incoming SOurce Dataframe

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([StructField("pk1", StringType(), True),
                    StructField("pk2", StringType(), True),
                    StructField("dim1", IntegerType(), True),
                    StructField("dim2", IntegerType(), True),
                    StructField("dim3", IntegerType(), True),
                    StructField("dim4", IntegerType(), True)])


In [0]:
data = [(111, 'Unit1', 200, 500, 800, 400),
    (222, 'Unit2', 800, 1300, 800, 500),
    (444, 'Unit4', 100, None, 700, 300)]

sourceDF = spark.createDataFrame(data, schema)
sourceDF.display()

pk1,pk2,dim1,dim2,dim3,dim4
111,Unit1,200,500,800,400
222,Unit2,800,1300,800,500
444,Unit4,100,null,700,300


### Approach: Left JOIN Source to Target

In [0]:
joinDF = sourceDF.join(targetDF, (sourceDF.pk1==targetDF.pk1) & \
                                    (sourceDF.pk2==targetDF.pk2) & \
                                    (targetDF.active_status=='Y'), "left") \
                                    .select(sourceDF["*"], \
                                    targetDF.pk1.alias("target_pk1"),\
                                    targetDF.pk2.alias("target_pk2"),\
                                    targetDF.dim1.alias("target_dim1"),\
                                    targetDF.dim2.alias("target_dim2"),\
                                    targetDF.dim3.alias("target_dim3"),\
                                    targetDF.dim4.alias("target_dim4")
                                    )

display(joinDF)

pk1,pk2,dim1,dim2,dim3,dim4,target_pk1,target_pk2,target_dim1,target_dim2,target_dim3,target_dim4
111,Unit1,200,500,800,400,111,Unit1,200,500,800,400
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100
444,Unit4,100,null,700,300,null,null,null,null,null,null


### Filter out Only Changed Records

In [0]:
filterDF = joinDF.filter(xxhash64(joinDF.dim1, joinDF.dim2, joinDF.dim3, joinDF.dim4)\
            != xxhash64(joinDF.target_dim1, joinDF.target_dim2, joinDF.target_dim3, joinDF.target_dim4))

filterDF.display()

pk1,pk2,dim1,dim2,dim3,dim4,target_pk1,target_pk2,target_dim1,target_dim2,target_dim3,target_dim4
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100
444,Unit4,100,null,700,300,null,null,null,null,null,null


### Create MERGEKEY by combining key columns

In [0]:
mergeDF = filterDF.withColumn("MERGEKEY", concat(filterDF.pk1, filterDF.pk2))
mergeDF.display()

pk1,pk2,dim1,dim2,dim3,dim4,target_pk1,target_pk2,target_dim1,target_dim2,target_dim3,target_dim4,MERGEKEY
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100,222Unit2
444,Unit4,100,null,700,300,null,null,null,null,null,null,444Unit4


### CREATE Null MERGEKEY only for matching records

In [0]:
dummyDF = filterDF.filter("target_pk1 IS NOT NULL").withColumn("MERGEKEY", lit(None))
dummyDF.display()

pk1,pk2,dim1,dim2,dim3,dim4,target_pk1,target_pk2,target_dim1,target_dim2,target_dim3,target_dim4,MERGEKEY
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100,null


In [0]:
scdDF = mergeDF.union(dummyDF)
scdDF.display()

pk1,pk2,dim1,dim2,dim3,dim4,target_pk1,target_pk2,target_dim1,target_dim2,target_dim3,target_dim4,MERGEKEY
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100,222Unit2
444,Unit4,100,null,700,300,null,null,null,null,null,null,444Unit4
222,Unit2,800,1300,800,500,222,Unit2,900,null,700,100,null


In [0]:
targetTable.alias("target").merge(
    source = scdDF.alias("source"),
    condition = "concat(target_pk1, target_pk2) = source.MERGEKEY and target.active_status = 'Y'"
).whenMatchedUpdate(set = {
    "active_status": "'N'",
    "end_date": "current_date"
}).whenNotMatchedInsert(values = {
    "pk1": "source.pk1",
    "pk2": "source.pk2",
    "dim1": "source.dim1",
    "dim2": "source.dim2",
    "dim3": "source.dim3",
    "dim4": "source.dim4",
    "active_status": "'Y'",
    "start_date": "current_date",
    "end_date": """to_date('9999-12-31', 'yyyy-MM-dd')"""
}).execute()

In [0]:
%sql
SELECT * FROM scd2demo

pk1,pk2,dim1,dim2,dim3,dim4,active_status,start_date,end_date
111,Unit1,200,500,800,400,N,2025-11-29T15:48:33.339+0000,2025-11-29T00:00:00.000+0000
222,Unit2,900,null,700,100,N,2025-11-29T15:48:33.339+0000,2025-11-29T00:00:00.000+0000
333,Unit3,300,900,250,650,N,2025-11-29T15:48:33.339+0000,2025-11-29T00:00:00.000+0000
222,Unit2,800,1300,800,500,Y,2025-11-29T00:00:00.000+0000,9999-12-31T00:00:00.000+0000
444,Unit4,100,null,700,300,Y,2025-11-29T00:00:00.000+0000,9999-12-31T00:00:00.000+0000
